In [1]:
import mysql.connector
import pandas as pd
import numpy as np

pd.options.display.max_columns = 999

In [2]:
def column_to_lowercase(df):
    for col in df.columns: 
        df = df.rename(columns={col:col.replace(' ', '_').lower()})
    return df


def fill_with_median(df, column_name):
    for i in df.index:
        if pd.isna(df.loc[i, column_name]):
            prev = df.loc[:i, column_name].dropna().last_valid_index()
            next = df.loc[i:, column_name].dropna().first_valid_index()
            if pd.notna(prev) and pd.notna(next):
                df.loc[i, column_name] = np.median([df.loc[prev, column_name], df.loc[next, column_name]])
    


# load data

In [3]:
# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="pass1234",
    database="sys"
)

cursor = connection.cursor()

# query to merge trips and stations
query = """
    SELECT 
        trips.*, 
        start_stations.id AS start_id, 
        start_stations.name AS start_name, 
        start_stations.lat AS start_lat, 
        start_stations.long AS start_lon,
        start_stations.zip as start_zip,
        start_stations.dock_count AS start_dock_count,
        
        end_stations.id AS end_id, 
        end_stations.name AS end_name, 
        end_stations.lat AS end_lat, 
        end_stations.long AS end_lon,
        end_stations.zip as end_zip,
        end_stations.dock_count AS end_dock_count


    FROM 
        trips 
    JOIN 
        stations AS start_stations 
    ON 
        trips.start_station = start_stations.id 
    JOIN 
        stations AS end_stations 
    ON 
        trips.end_station = end_stations.id;
"""
trips_x_stations = pd.read_sql(query, connection)

# column to lowercase
trips_x_stations = column_to_lowercase(trips_x_stations)

# cast to datetime the column
trips_x_stations['start_date'] = pd.to_datetime(trips_x_stations['start_date'], dayfirst=True)
trips_x_stations['end_date'] = pd.to_datetime(trips_x_stations['end_date'], dayfirst=True)
cursor.close()

print(trips_x_stations.shape)
trips_x_stations.head()

/var/folders/xj/k1z1n4v16h11nprq8vsfqn1r0000gq/T/ipykernel_72723/4225382094.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trips_x_stations = pd.read_sql(query, connection)


(354152, 18)


,trip_id,start_date,start_station,end_date,end_station,subscriber_type,start_id,start_name,start_lat,start_lon,start_zip,start_dock_count,end_id,end_name,end_lat,end_lon,end_zip,end_dock_count
0,913459,2015-08-31 23:11:00,31,2015-08-31 23:28:00,27,Subscriber,31,San Antonio Shopping Center,37.400443,-122.108338,94041,15,27,Mountain View City Hall,37.389218,-122.081896,94041,15
1,913454,2015-08-31 23:10:00,10,2015-08-31 23:17:00,8,Subscriber,10,San Jose City Hall,37.337391,-121.886995,95113,15,8,San Salvador at 1st,37.330165,-121.885831,95113,15
2,913273,2015-08-31 19:08:00,22,2015-08-31 19:12:00,26,Subscriber,22,Redwood City Caltrain Station,37.486078,-122.232089,94063,25,26,Redwood City Medical Center,37.487682,-122.223492,94063,15
3,913223,2015-08-31 18:45:00,80,2015-08-31 18:57:00,6,Subscriber,80,Santa Clara County Civic Center,37.352601,-121.905733,95113,15,6,San Pedro Square,37.336721,-121.894074,95113,15
4,913217,2015-08-31 18:43:00,2,2015-08-31 18:56:00,16,Subscriber,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,95113,27,16,SJSU - San Salvador at 9th,37.333955,-121.877349,95113,15


In [4]:
# load and cast weather
weather = pd.read_csv('./data/weather_data.csv')
weather = column_to_lowercase(weather)
weather['date'] = pd.to_datetime(weather['date'], dayfirst=True)
print(weather.shape)
weather.head()

(1825, 24)


,date,max_temperaturef,mean_temperaturef,min_temperaturef,max_dew_pointf,meandew_pointf,min_dewpointf,max_humidity,mean_humidity,min_humidity,max_sea_level_pressurein,mean_sea_level_pressurein,min_sea_level_pressurein,max_visibilitymiles,mean_visibilitymiles,min_visibilitymiles,max_wind_speedmph,mean_wind_speedmph,max_gust_speedmph,precipitationin,cloudcover,events,winddirdegrees,zip
0,2014-09-01,83.0,70.0,57.0,58.0,56.0,52.0,86.0,64.0,42.0,29.86,29.82,29.76,10.0,10.0,8.0,16.0,7.0,20.0,0.0,0.0,NaN,290.0,94107
1,2014-09-02,72.0,66.0,60.0,58.0,57.0,55.0,84.0,73.0,61.0,29.87,29.82,29.79,10.0,10.0,7.0,21.0,8.0,NaN,0.0,5.0,NaN,290.0,94107
2,2014-09-03,76.0,69.0,61.0,57.0,56.0,55.0,84.0,69.0,53.0,29.81,29.76,29.72,10.0,10.0,10.0,21.0,8.0,24.0,0.0,4.0,NaN,276.0,94107
3,2014-09-04,74.0,68.0,61.0,57.0,57.0,56.0,84.0,71.0,57.0,29.81,29.76,29.72,10.0,10.0,8.0,22.0,8.0,25.0,0.0,5.0,NaN,301.0,94107
4,2014-09-05,72.0,66.0,60.0,57.0,56.0,54.0,84.0,71.0,57.0,29.92,29.87,29.81,10.0,9.0,7.0,18.0,8.0,32.0,0.0,4.0,NaN,309.0,94107


# manage missing data in weateher

In [5]:
weather = weather.sort_values(by='date')

# by intution, filled with sunny
weather['events'] = weather['events'].fillna('sunny')

# compute the columns with nans
columns_with_nans = [c for c in weather.columns if weather[weather[c].isna()].shape[0] > 0  ]

# creaete a dict for each zip
zip_dict = {zzip: weather[weather['zip'] == zzip] for zzip in weather.zip.unique()}

# for each zip, column with nan fill the nans with median between the two closest value to the nan
# assumption: weather data could not vary so widley in a limted amount of time
for key in zip_dict.keys():
    for col_with_nans in columns_with_nans:
        fill_with_median(zip_dict[key], col_with_nans)
weather = pd.concat(zip_dict.values())

# some columns are not properlu filled up, so they are removed
col_still_with_nans = [col for col in weather.columns if weather[weather[col].isna()].shape[0] > 0  ]
weather = weather.drop(col_still_with_nans, axis=1)

for col in weather.columns:
    assert weather[weather[col].isna()].shape[0] == 0


In [6]:
# to merge without asof, i extract the day date and cast int a string
trips_x_stations['start_date_day'] = trips_x_stations['start_date'].dt.date.astype(str)
trips_x_stations['end_date_day'] = trips_x_stations['end_date'].dt.date.astype(str)
weather['date_merge'] = weather['date'].dt.date.astype(str)

# Transform the trips into flows

In [7]:
# stringfication of date hour
trips_x_stations['date_hour'] = trips_x_stations.start_date.dt.date.astype(str) + ' ' + trips_x_stations.start_date.dt.hour.astype(str) +':00:00'
# counting all trips leaving at each hour for each statons
start_per_date_per_loc = trips_x_stations.groupby(['date_hour' ,'start_station', 'start_zip']).count().trip_id.reset_index()
# coliumn standardization
start_per_date_per_loc = start_per_date_per_loc.rename(columns = {'start_station':'station', 'trip_id':'trip_in', 'start_zip':'zip'})


# stame procedure for flow out
trips_x_stations['date_hour'] = trips_x_stations.start_date.dt.date.astype(str) + ' ' + trips_x_stations.start_date.dt.hour.astype(str) +':00:00'
end_per_date_per_loc   = trips_x_stations.groupby(['date_hour', 'end_station', 'end_zip']).count().trip_id.reset_index()
end_per_date_per_loc = end_per_date_per_loc.rename(columns = {'end_station':'station', 'trip_id':'trip_out', 'end_zip':'zip'})

# merge #bikes leaving and #bikes starting
flow_per_stat = start_per_date_per_loc.merge(
    end_per_date_per_loc,
    on=['date_hour', 'station', 'zip'],
)
# flow computaation
flow_per_stat['flow'] = flow_per_stat['trip_in'] - flow_per_stat['trip_out']

# date casting
flow_per_stat['date'] = pd.to_datetime(flow_per_stat['date_hour'])

# date stringication for further merges
flow_per_stat['date_merge'] = flow_per_stat['date'].dt.date.astype(str)
flow_per_stat = flow_per_stat.drop(['date_hour', 'trip_in', 'trip_out'], axis=1)

In [8]:
# # merge the weather features
flow_per_stat
df = flow_per_stat.merge(
    weather.drop('date', axis=1),
    on = ['date_merge', 'zip'],
)
df = df.drop(['date_merge'], axis=1)
df['date'] = pd.to_datetime(df['date'])

# # final_df
print('final shape', df.shape)

final shape (84167, 25)


In [9]:
df.head()

,station,zip,flow,date,max_temperaturef,mean_temperaturef,min_temperaturef,max_dew_pointf,meandew_pointf,min_dewpointf,max_humidity,mean_humidity,min_humidity,max_sea_level_pressurein,mean_sea_level_pressurein,min_sea_level_pressurein,max_visibilitymiles,mean_visibilitymiles,min_visibilitymiles,max_wind_speedmph,mean_wind_speedmph,precipitationin,cloudcover,events,winddirdegrees
0,41,94107,-1,2014-09-01 10:00:00,83.0,70.0,57.0,58.0,56.0,52.0,86.0,64.0,42.0,29.86,29.82,29.76,10.0,10.0,8.0,16.0,7.0,0.0,0.0,sunny,290.0
1,60,94107,-2,2014-09-01 10:00:00,83.0,70.0,57.0,58.0,56.0,52.0,86.0,64.0,42.0,29.86,29.82,29.76,10.0,10.0,8.0,16.0,7.0,0.0,0.0,sunny,290.0
2,65,94107,-1,2014-09-01 10:00:00,83.0,70.0,57.0,58.0,56.0,52.0,86.0,64.0,42.0,29.86,29.82,29.76,10.0,10.0,8.0,16.0,7.0,0.0,0.0,sunny,290.0
3,67,94107,1,2014-09-01 10:00:00,83.0,70.0,57.0,58.0,56.0,52.0,86.0,64.0,42.0,29.86,29.82,29.76,10.0,10.0,8.0,16.0,7.0,0.0,0.0,sunny,290.0
4,73,94107,4,2014-09-01 10:00:00,83.0,70.0,57.0,58.0,56.0,52.0,86.0,64.0,42.0,29.86,29.82,29.76,10.0,10.0,8.0,16.0,7.0,0.0,0.0,sunny,290.0


# adding new features

In [10]:
# is weekend
df['is_weekend'] = df.date.dt.day_of_week >= 5
df['is_workingday'] = df.date.dt.day_of_week < 5

# one hot encodinf of string colums
df = pd.get_dummies(df, columns=['events'])

# # #compute flow lags per pount

def calculate_previous_flow(df, delta_hours=1):
    # Assicurati che 'date' sia di tipo datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Imposta 'station' e 'date' come multi-indice
    df = df.set_index(['station', 'date'])
    
    # Ordina l'indice per assicurare che il shift operi correttamente
    df.sort_index(inplace=True)
    
    # Applica shift dentro ogni gruppo di stazione
    df[f'last_{delta_hours}_flow'] = df.groupby(level='station')['flow'].shift(delta_hours)
    
    # Reset dell'indice
    df = df.reset_index()
    
    return df


df = calculate_previous_flow(df, 1)
df = calculate_previous_flow(df, 2)
df = calculate_previous_flow(df, 4)
df = calculate_previous_flow(df, 5)



df['hour'] = df['date'].dt.hour
df['dow'] = df['date'].dt.day_of_week
df['month'] = df['date'].dt.month
df['woy'] = df['date'].dt.isocalendar().week





In [11]:
validation_set = df[
    (df.date >= pd.to_datetime('2015-08-01')) &
    (df.date < pd.to_datetime('2015-09-01'))
]
validation_set.to_csv('./processed_data/validation_set.csv', index=False)

train_and_test = df[(df.date < pd.to_datetime('2015-08-01'))]
train_and_test.to_csv('./processed_data/train_and_test.csv', index=False)

In [12]:
# grp = df.groupby([df.start_date.dt.date, 'start_id']).count().trip_id.reset_index()

# for stat in grp.start_id:
#     grp[grp == stat].trip_id.plot()